In [245]:
""" import libraries """
from urllib.request import urlopen
from bs4 import BeautifulSoup
import lxml
import requests
import re
import pandas as pd

In [67]:
"""initialise a session""" 

# load the landing page
landing_page = "https://www.bbc.co.uk/programmes/b006qnmr/episodes/player?page={}"
html = requests.get(landing_page.format(1))
soup = BeautifulSoup(html.content, 'html.parser')

# now find the number of pages
npages = int(soup.find('li', {"class": "pagination__page--last"}).find('a').text)
print(npages)

231


In [116]:
""" run through the pages and fetch title and url_list """ 
title_list = []
title_url_list = []

for page in range(1, npages+1):
    html = requests.get(landing_page.format(page))
    soup = BeautifulSoup(html.content, 'html.parser')
    
    for title in soup.find_all('h2', class_='programme__titles'):
        title_list.append(title.text)
        for href in title:
            title_url_list.append(href['href'])
        print((page)/(npages + 1), end='\r')

In [249]:
""" now run through the different episodes """ 

DF = pd.DataFrame(columns = ['guest', 'artist', 'track', 'label', 'presenter', 'producer', 'book_choice', 'luxury_item', 'favourite', 'synopsis'])
for ep in range(1, len(title_url_list)+1):
    html = requests.get(title_url_list[0])
    soup = BeautifulSoup(html.content, 'html.parser')

    for entry in soup.find_all('p'):
        if 'Presenter' in entry.text:
            [presenter, producer] = entry.text[11:].split('Producer: ')
    #         print(f"Presenter was: {Presenter}, Producer was: {Producer}")
            break

    for entry in soup.find_all('p'):
        if 'BOOK CHOICE' in entry.text:
            [book_choice, luxury_item, favourite] = re.split("LUXURY ITEM: |CASTAWAY'S FAVOURITE: ", entry.text[13:])
            break

    synopsis = ""
    for entry in soup.find_all(class_='synopsis-toggle__long'):
        synopsis += entry.text
    
    for segment in soup.find_all(class_='segment segment--music'):
        artist = segment.find(class_='artist').text
        track = segment.find_all(class_='no-margin')[1].find('span').text
        label = segment.find('abbr').text
        DF = DF.append({'guest': title_list[ep-1]
                          'artist': artist, 
                          'track': track, 
                          'label':label, 
                          'presenter':presenter, 
                          'producer':producer, 
                          'book_choice':book_choice, 
                          'luxury_item':luxury_item, 
                          'favourite':favourite, 
                          'synopsis':synopsis}, ignore_index=True)
        
    print((ep)/(len(title_url_list) + 1), end='\r')
        

In [259]:
DF.describe()

,guest,artist,track,label,presenter,producer,book_choice,luxury_item,favourite,synopsis
count,0,18472,18472,18472,18472,18472,18472,18472,18472,18472
unique,0,8,8,7,1,1,1,1,1,1
top,NaN,John Legend,It Is Well With My Soul,Sony Music Entertainment UK Ltd.,Lauren Laverne,Paula McGinley,Who Moved My Cheese? by Dr Spencer Johnson,Quality Street chocolates,It Is Well With My Soul by Wintley Phipps,\nAmanda Khozi Mukwashi is the chief executive...
freq,NaN,2309,2309,4618,18472,18472,18472,18472,18472,18472
